coding: utf-8


Registration using optical flow<br>
==================================<br>
Demonstration of image registration using optical flow.<br>
By definition, the optical flow is the vector field *(u, v)* verifying<br>
*image1(x+u, y+v) = image0(x, y)*, where (image0, image1) is a couple of<br>
consecutive 2D frames from a sequence. This vector field can then be<br>
used for registration by image warping.<br>
To display registration results, an RGB image is constructed by<br>
assigning the result of the registration to the red channel and the<br>
target image to the green and blue channels. A perfect registration<br>
results in a gray level image while misregistred pixels appear colored<br>
in the constructed RGB image.<br>


In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from skimage.color import rgb2gray
from skimage.data import stereo_motorcycle, vortex
from skimage.transform import warp
from skimage.registration import optical_flow_tvl1, optical_flow_ilk

--- Load the sequence

In [ ]:
image0, image1, disp = stereo_motorcycle()

--- Convert the images to gray level: color is not supported.

In [ ]:
image0 = rgb2gray(image0)
image1 = rgb2gray(image1)

--- Compute the optical flow

In [ ]:
v, u = optical_flow_tvl1(image0, image1)

--- Use the estimated optical flow for registration

In [ ]:
nr, nc = image0.shape

In [ ]:
row_coords, col_coords = np.meshgrid(np.arange(nr), np.arange(nc),
                                     indexing='ij')

In [ ]:
image1_warp = warp(image1, np.array([row_coords + v, col_coords + u]),
                   mode='edge')

build an RGB image with the unregistered sequence

In [ ]:
seq_im = np.zeros((nr, nc, 3))
seq_im[..., 0] = image1
seq_im[..., 1] = image0
seq_im[..., 2] = image0

build an RGB image with the registered sequence

In [ ]:
reg_im = np.zeros((nr, nc, 3))
reg_im[..., 0] = image1_warp
reg_im[..., 1] = image0
reg_im[..., 2] = image0

build an RGB image with the registered sequence

In [ ]:
target_im = np.zeros((nr, nc, 3))
target_im[..., 0] = image0
target_im[..., 1] = image0
target_im[..., 2] = image0

--- Show the result

In [ ]:
fig, (ax0, ax1, ax2) = plt.subplots(3, 1, figsize=(5, 10))

In [ ]:
ax0.imshow(seq_im)
ax0.set_title("Unregistered sequence")
ax0.set_axis_off()

In [ ]:
ax1.imshow(reg_im)
ax1.set_title("Registered sequence")
ax1.set_axis_off()

In [ ]:
ax2.imshow(target_im)
ax2.set_title("Target")
ax2.set_axis_off()

In [ ]:
fig.tight_layout()

#################################################################<br>
The estimated vector field *(u, v)* can also be displayed with a<br>
quiver plot.<br>
<br>
In the following example, Iterative Lukas-Kanade algorithm (iLK) is<br>
applied to images of particles in the context of particle image<br>
velocimetry (PIV). The sequence is the Case B from the<br>
`PIV challenge 2001 <http://www.pivchallenge.org/>`_

In [ ]:
image0, image1 = vortex()

--- Compute the optical flow

In [ ]:
v, u = optical_flow_ilk(image0, image1, radius=15)

--- Compute flow magnitude

In [ ]:
norm = np.sqrt(u ** 2 + v ** 2)

--- Display

In [ ]:
fig, (ax0, ax1) = plt.subplots(1, 2, figsize=(8, 4))

--- Sequence image sample

In [ ]:
ax0.imshow(image0, cmap='gray')
ax0.set_title("Sequence image sample")
ax0.set_axis_off()

--- Quiver plot arguments

In [ ]:
nvec = 20  # Number of vectors to be displayed along each image dimension
nl, nc = image0.shape
step = max(nl//nvec, nc//nvec)

In [ ]:
y, x = np.mgrid[:nl:step, :nc:step]
u_ = u[::step, ::step]
v_ = v[::step, ::step]

In [ ]:
ax1.imshow(norm)
ax1.quiver(x, y, u_, v_, color='r', units='dots',
           angles='xy', scale_units='xy', lw=3)
ax1.set_title("Optical flow magnitude and vector field")
ax1.set_axis_off()
fig.tight_layout()

In [ ]:
plt.show()